In [4]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt, style; style.use('fivethirtyeight')
import seaborn as sns; sns.set(context='talk')

from IPython.display import display, HTML, IFrame

Dataset Exploration

In [2]:
df = pd.read_csv('./data/twtc.csv')

df_label_mask = (df.drop(columns='label') == 0).assign(label=False)
df = df.mask(df_label_mask)

df.describe().T[['mean', 'std', 'min', '50%', 'max']]

,mean,std,min,50%,max
Unnamed: 0,4587.500000,2648.450018,1.00,4587.5,9174.00
age,20.810431,2.307676,15.30,21.0,31.90
year,2016.715095,1.979656,2013.00,2017.0,2019.00
eta,2018.802799,2.422862,2013.00,2019.0,2025.00
Arm,53.824127,6.897962,30.00,55.0,80.00
Changeup,49.858290,5.364976,30.00,50.0,70.00
Control,49.205290,5.059892,30.00,50.0,70.00
Curveball,52.929583,5.675287,35.00,55.0,70.00
Cutter,52.475000,4.936723,40.00,50.0,70.00
Fastball,59.531873,6.665786,40.00,60.0,80.00


In [3]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, BertTokenizer

In [2]:
labeled = pd.read_csv('data/labeled_scouting.csv')
test = pd.read_csv('data/augment/test.csv')
train = pd.concat([labeled, test]).drop_duplicates(keep=False)

In [5]:
# encoded_corpus = tokenizer(text=train['text'].tolist(),
#                             add_special_tokens=True,
#                             padding='max_length',
#                             truncation='longest_first',
#                             max_length=300,
#                             return_attention_mask=True)

def tokenize_data(df, tokenizer):
    tokenized_df = tokenizer(df['text'].tolist(), 
                                add_special_tokens=True,
                                truncation=True, 
                                padding="max_length",
                                return_attention_mask=True)

    input_ids = np.array(tokenized_df['input_ids'])
    attention_mask = np.array(tokenized_df['attention_mask'])
    return input_ids, attention_mask, np.array(df['label'])


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_input_ids, train_attention_mask, train_labels = tokenize_data(train, tokenizer)
test_input_ids, test_attention_mask, test_labels = tokenize_data(test, tokenizer)
                                

In [12]:
print(train['text'].iloc[0])

print(train_input_ids[0])


PERSON is a Level 1 sex offender and wouldn't be on our board were we running a draft room. The general consensus around the industry is that a team is going to draft PERSON, so we're compelled to cover his ability. We have him ranked where we think he fits on the continuum of talent, early in round two, though he probably won't be drafted until much later. PERSON was projected to go early in round two last year, when he was a junior. Shortly before the draft, The GPE reported court documents that showed PERSON plead guilty to sexually assaulting his niece. Court records showed the victim reported multiple incidents of molestation between 2009 and 2011, when PERSON was 14-15 years old and the victim was 4-6. He plead guilty to one count which included a handwritten admission and the the other count was dismissed as part of a plea bargain. After this information surface, PERSON spent the rest of the spring of 2017 away from GPE State and went undrafted. He returned for his senior season

In [14]:
X_train = np.random.randint(-1,3, size=(3, 3))
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)

array([[-0.70710678,  0.70710678, -1.22474487],
       [ 1.41421356, -1.41421356,  0.        ],
       [-0.70710678,  0.70710678,  1.22474487]])

In [ ]:
def compute_mixup(a, b, l):
    return l * np.array(a) + (1 - l) * np.array(b)


def mixup(row1, row2, l):
    mixed_val = list(map(int, compute_mixup(row1['input_ids'].iloc[0], row2['input_ids'].iloc[0], l)))
    mixed_label = compute_mixup(row1['label'], row2['label'], l)

    if sum(row1['attention_mask'].iloc[0]) > sum(row2['attention_mask'].iloc[0]):
        new_attention = row1['attention_mask'].iloc[0]
    else:
        new_attention = row2['attention_mask'].iloc[0]

    return {
        'label': mixed_label,
        'input_ids': mixed_val,
        'token_type_ids': row1['token_type_ids'].iloc[0],
        'attention_mask': new_attention
    }

def mixup_aug(input_ids, att_masks, labels, lam, n=5):
    mixed_input_ids = []
    mixed_att_masks = []
    mixed_labels = []
    
    for i in range(n):
        a = np.random.randint(len(input_ids))
        b = np.random.randint(len(input_ids))
        
        x = list(map(int, compute_mixup(input_ids[a], input_ids[b], lam)))
        print(x)
        
    
        rows.append(mixup(row1, row2, lam))
    tokenized_train = pd.DataFrame(rows)
    shuffle(tokenized_train)
    return Dataset.from_pandas(tokenized_train)

In [51]:
mixup_aug(train_input_ids, train_attention_mask, train_labels, 0.4)

[101, 2465, 2078, 2603, 1406, 4222, 1010, 2403, 2167, 2192, 2187, 1616, 17840, 2037, 8931, 2172, 2556, 3986, 2534, 1422, 2061, 2601, 2035, 2171, 2951, 2182, 9378, 5133, 1517, 2341, 1673, 3033, 1010, 2203, 2005, 2707, 5175, 5256, 2090, 5759, 12313, 1607, 2102, 5309, 9893, 1601, 2232, 1416, 9345, 1406, 3842, 5262, 4701, 4159, 2053, 2047, 2070, 2211, 1606, 2011, 1643, 1670, 1429, 6326, 3602, 3301, 2393, 2065, 2199, 2221, 2007, 2618, 1705, 2056, 2454, 2292, 2089, 7359, 3180, 2026, 2429, 1405, 2674, 1998, 2202, 2250, 3833, 3575, 2411, 5378, 8931, 1421, 1404, 2432, 1606, 1436, 7176, 2321, 2951, 1627, 1612, 2261, 2165, 1027, 2519, 2909, 7473, 1980, 5092, 4392, 4776, 4181, 2038, 4780, 2323, 4236, 1850, 2410, 13567, 2774, 2862, 3167, 1416, 8010, 7018, 2059, 3156, 2050, 2413, 4565, 4433, 1401, 4664, 9086, 2473, 1612, 2051, 2114, 1613, 2013, 1404, 2352, 1420, 2352, 13936, 4213, 1998, 1420, 2703, 1603, 1423, 9789, 1021, 5122, 3115, 1611, 2017, 3887, 4323, 2002, 1461, 7028, 2008, 3791, 2379, 1562, 

NameError: name 'rows' is not defined

In [42]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 16

def create_dataloaders(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor, 
                            labels_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, 
                            shuffle=True)
    return dataloader

train_dataloader = create_dataloaders(train_input_ids, train_attention_mask, 
                                      train_labels, batch_size)
test_dataloader = create_dataloaders(test_input_ids, test_attention_mask, 
                                     test_labels, batch_size)

In [13]:
mixup_aug(train, 0.5)

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10
})

In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

arguments = TrainingArguments(
    output_dir="sample_SD_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)


def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=train,
    eval_dataset=test, # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
from transformers import TrainerCallback, EarlyStoppingCallback

class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0))
trainer.add_callback(LoggingCallback("sample_SD_trainer/log.jsonl"))

In [ ]:
trainer.train()

/Users/amolsingh/anaconda3/envs/local_nmt/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6222
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1167


Epoch,Training Loss,Validation Loss


From_Pretrained

In [ ]:
# To load our saved model, we can pass the path to the checkpoint into the `from_pretrained` method:
test_str = "I enjoyed the movie!"

finetuned_model = AutoModelForSequenceClassification.from_pretrained("sample_SD_trainer/checkpoint-24")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(finetuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])